# Prepare data

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
df = pd.read_csv('/kaggle/input/real-time-advertisers-auction/Dataset.csv')
df.date = pd.to_datetime(df.date)

In [ ]:
#calculating CPM from https://www.kaggle.com/akshaypaliwal709/predicting-the-reserve-price-base-cpm
#calculating the value that the Advertisers Bid for the month of June
# CPM(the value which was the winning bid value) = 
#((revenue of the publisher*100)/revenue_share_percentage)/measurable_impressions)*1000

def weird_division(n, d):
    return n / d if d else 0

df['CPM'] = df.apply(lambda x: weird_division(((x['total_revenue']*100)),x['measurable_impressions'])*1000 , axis=1)

In [ ]:
# Cut negative values, values > 95 percentile
cpm_95_quantile = df.CPM.quantile(0.95)
df = df[(df.CPM>=0) & (df.CPM<cpm_95_quantile)]

In [ ]:
print(f"Delete if is constant: revenue_share_percent - \
{df.revenue_share_percent.nunique()==1}, integration_type_id - {df.integration_type_id.nunique()==1}")
df.drop(columns=["revenue_share_percent","integration_type_id"], inplace=True)

# Also delete target-related features
df.drop(columns=["total_revenue","measurable_impressions","total_impressions","viewable_impressions"], inplace=True)

In [ ]:
df.columns

In [ ]:
# df.hist(figsize=(15,15))

In [ ]:
TEST_DT = datetime(2019,6,22)
pd_train = df[df.date < TEST_DT]
pd_test = df[df.date >= TEST_DT]
print(f"train: {pd_train.date.min()}-{pd_train.date.max()}, test: {pd_test.date.min()}-{pd_test.date.max()}")
X_train = pd_train.drop(columns=["date","CPM"])
y_train = pd_train.CPM
X_test = pd_test.drop(columns=["date","CPM"])
y_test = pd_test.CPM

# Model

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error as mse

In [ ]:
lgb = LGBMRegressor()
lgb.fit(X_train, y_train)

In [ ]:
y_predict = lgb.predict(X_test)

In [ ]:
mse(y_test, y_predict)